<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-25 10:23:40
-------------------
qualified stocks: 93
with latest results: 27
still star stocks: 17
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  3.39 L
Current:  1.37 C
-------------------
Today PnL: 23.31 K (0.17%)
Current PnL: -17.88 L (-12.16%)
CY Booked + Current PnL: -9.15 L (-6.22%)
-------------------
Total profit:  3.63 L
Total loss:  -21.51 L
-------------------
Total Booked + Current PnL: 17.83 L (14.85%)
Total Booked PnL: 35.71 L (29.75%)
Curr Year Booked PnL: 8.73 L (6.35%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.14 C
Est FTT PnL: 76.99 L (56.02%)
Deployed:  1.20 C
Current:  1.37 C
CAGR/XIRR %: 8.58%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,HINDALCO,651.95,756.01,-17.42,H-LC,7.18,114288.0,9976.0,6674.0,1.43,9.56,5.84,15.96,9.0,1.49,0.85,28.08,X5K,ATH,METALS
4,APOLLOHOSP,7098.95,8285.00,-13.42,H-LC,7.88,172964.0,16787.0,9305.0,-0.76,10.75,5.38,16.71,22.0,1.80,1.29,30.06,X40N,BTT,HEALTHCARE
55,MEDANTA,1087.93,1486.00,12.96,H-MC,14.67,161344.0,35144.0,11036.0,0.45,27.85,6.84,36.59,108.0,3.18,1.20,43.90,XY24,NTT,HEALTHCARE
37,HINDUNILVR,2413.81,2723.00,-6.58,H-LC,7.72,293664.0,23317.0,11306.0,-0.30,8.62,3.85,12.81,24.0,2.06,2.19,22.82,XY25,NTT,FMCG
68,SBIN,760.30,863.00,-12.30,M-LC,3.95,216426.0,14946.0,12271.0,0.06,7.42,5.67,13.51,60.0,1.22,1.61,20.95,XY25,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,116.91,190.00,-30.43,H-MC,7.28,178131.0,-1677.0,114093.0,0.31,-0.93,64.05,62.52,88.0,-0.01,1.33,32.84,XR,NTT,BANKS
58,PGHH,13388.00,18062.58,-31.08,H-MC,5.53,199320.0,-1500.0,71616.0,-0.14,-0.75,35.93,34.92,80.0,-0.02,1.49,3.89,X40,ATH,FMCG


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,ITC,411.90,452.0,-39.61,H-LC,2.99,144672.0,-5260.0,19849.0,-0.21,-3.51,13.72,9.74,4.0,-0.27,1.08,2.51,X40,NTT,FMCG
53,LTIM,5564.16,7230.2,-3.47,H-LC,3.37,191718.0,-8592.0,68578.0,1.93,-4.29,35.77,29.94,16.0,-0.13,1.43,32.45,X200,ATH,IT
57,NESTLEIND,1134.30,1377.0,-11.66,H-LC,3.54,271674.0,6248.0,50531.0,-0.16,2.35,18.60,21.40,11.0,0.12,2.03,9.61,XY25,NTT,FMCG
21,CIPLA,1492.70,1795.0,-17.72,H-LC,4.19,219611.0,15111.0,26309.0,0.64,7.39,11.98,20.25,10.0,0.57,1.64,16.77,X40N,BTT,PHARMA
5,ASIANPAINT,2961.56,4250.0,-12.67,H-LC,5.64,170306.0,-31080.0,118686.0,-2.42,-15.43,69.69,43.51,27.0,-0.26,1.27,18.55,X40,BTT,PAINTS


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TCS,3794.03,4998.00,-26.12,H-LC,16.11,244569.0,-51365.0,145274.0,2.67,-17.36,59.40,31.73,1.0,-0.35,1.82,4.41,X40,BTT,IT
45,INFY,1461.46,2275.00,-16.73,H-LC,12.01,272975.0,11374.0,134249.0,2.52,4.35,49.18,55.67,3.0,0.08,2.04,10.72,X40,BTT,IT
17,BSOFT,628.85,836.99,-12.86,H-SC,16.23,67174.0,-42875.0,79299.0,2.25,-38.96,118.05,33.10,131.0,-0.54,0.50,11.62,XR,ATH,IT
63,REPCOHOME,515.07,880.00,-59.76,H-SC,6.44,148076.0,-56922.0,202168.0,2.00,-27.77,136.53,70.85,134.0,-0.28,1.10,21.65,XY24,NTT,FINANCE
34,HCLTECH,1562.67,1943.91,2.80,H-LC,10.58,182390.0,-8256.0,54772.0,1.96,-4.33,30.03,24.40,8.0,-0.15,1.36,10.92,X40,ATH,IT


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
29,EASEMYTRIP,18.12,26.40,-9.31,M-SC,27.20,87334.0,-89481.0,170275.0,-3.97,-50.61,194.97,45.70,197.0,-0.53,0.65,1.59,XY24,NTT,TRAVEL
5,ASIANPAINT,2961.56,4250.00,-12.67,H-LC,5.64,170306.0,-31080.0,118686.0,-2.42,-15.43,69.69,43.51,27.0,-0.26,1.27,18.55,X40,BTT,PAINTS
24,COLPAL,2714.63,3767.14,-1.47,H-MC,11.82,179345.0,-32396.0,114494.0,-2.18,-15.30,63.84,38.77,84.0,-0.28,1.34,6.75,XY25,ATH,FMCG
23,COFFEEDAY,59.14,80.00,-43.66,L-SC,25.79,83309.0,-30240.0,70288.0,-2.12,-26.63,84.37,35.27,268.0,-0.43,0.62,103.90,XR,NTT,HOTELS
6,ASIANTILES,75.41,137.00,6816.67,L-SC,6.81,75884.0,-17926.0,94544.0,-1.69,-19.11,124.59,81.67,269.0,-0.19,0.57,46.95,XR,NTT,CERAMICS


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TTKPRESTIG,769.29,770.0,100.85,M-SC,8.84,86565.0,-14212.0,14309.0,0.96,-14.10,16.53,0.09,245.0,-0.99,0.65,13.73,OX40N,NTT,DURABLES
84,UJJIVANSFB,52.77,60.0,87.86,H-SC,15.73,118395.0,-24084.0,43605.0,0.32,-16.90,36.83,13.70,163.0,-0.55,0.88,39.74,OX40N,NTT,BANKS
20,CERA,7989.07,9475.0,-18.38,H-SC,3.08,116415.0,-27388.0,54133.0,-0.93,-19.05,46.50,18.60,149.0,-0.51,0.87,28.47,OX40N,NTT,CERAMICS
43,INDIGOPNTS,1407.73,1408.0,113.73,M-SC,1.81,140046.0,-34513.0,34549.0,-0.36,-19.77,24.67,0.02,221.0,-1.00,1.04,21.82,OX40N,NTT,PAINTS
72,SIS,477.00,528.0,2240.93,H-SC,5.64,62985.0,-18105.0,26775.0,1.30,-22.33,42.51,10.69,156.0,-0.68,0.47,27.65,OX40N,NTT,MISC


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TCS,3794.03,4998.00,-26.12,H-LC,16.11,244569.0,-51365.0,145274.0,2.67,-17.36,59.40,31.73,1.0,-0.35,1.82,4.41,X40,BTT,IT
45,INFY,1461.46,2275.00,-16.73,H-LC,12.01,272975.0,11374.0,134249.0,2.52,4.35,49.18,55.67,3.0,0.08,2.04,10.72,X40,BTT,IT
47,ITC,411.90,452.00,-39.61,H-LC,2.99,144672.0,-5260.0,19849.0,-0.21,-3.51,13.72,9.74,4.0,-0.27,1.08,2.51,X40,NTT,FMCG
89,VBL,492.64,671.64,-8.30,H-LC,8.95,278630.0,13590.0,82725.0,0.25,5.13,29.69,36.33,5.0,0.16,2.08,19.11,X40N,ATH,FMCG
1,ABB,5551.76,7934.00,-41.62,H-LC,12.58,198256.0,-18263.0,111162.0,0.40,-8.43,56.07,42.91,7.0,-0.16,1.48,3.83,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.0,12.96,H-MC,14.67,161344.0,35144.0,11036.0,0.45,27.85,6.84,36.59,108.0,3.18,1.20,43.90,XY24,NTT,HEALTHCARE
28,DMART,3764.92,5201.0,-7.19,H-LC,10.30,231770.0,47289.0,23084.0,0.66,25.63,9.96,38.14,38.0,2.05,1.73,38.96,X40N,NTT,FMCG


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.00,12.96,H-MC,14.67,161344.0,35144.0,11036.0,0.45,27.85,6.84,36.59,108.0,3.18,1.20,43.90,XY24,NTT,HEALTHCARE
15,BLUESTARCO,1646.70,2326.38,11.16,H-MC,5.01,192000.0,27330.0,40646.0,0.00,16.60,21.17,41.28,89.0,0.67,1.43,25.90,X40N,ATH,AC
51,KPIGREEN,497.21,731.35,9.02,H-SC,10.92,129881.0,4584.0,54420.0,-0.66,3.66,41.90,47.09,141.0,0.08,0.97,62.56,MH,ATH,POWER
90,VOLTAS,1278.28,1918.49,-2.06,H-MC,6.94,205500.0,13758.0,82282.0,1.24,7.18,40.04,50.08,99.0,0.17,1.53,14.46,XY25,ATH,AC
19,CAMS,3643.00,5211.76,-4.05,H-SC,5.80,109200.0,7196.0,36724.0,-0.13,7.05,33.63,43.06,122.0,0.20,0.81,25.38,X40N,ATH,MISC


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-4.42,H-MC,13.13,176946.0,26033.0,31266.0,1.15,17.25,17.67,37.97,93.0,0.83,1.32,46.92,AR,ATH,AUTO
42,INDIAMART,2327.09,4850.92,-49.41,H-SC,12.31,141123.0,17787.0,115975.0,-0.29,14.42,82.18,108.45,119.0,0.15,1.05,40.00,AR,ATH,MISC
91,WHIRLPOOL,1219.98,2270.00,-42.05,M-SC,5.50,97972.0,6474.0,72274.0,0.41,7.07,73.77,86.07,223.0,0.09,0.73,41.73,XR,NTT,DURABLES
51,KPIGREEN,497.21,731.35,9.02,H-SC,10.92,129881.0,4584.0,54420.0,-0.66,3.66,41.90,47.09,141.0,0.08,0.97,62.56,MH,ATH,POWER
61,RAJOOENG,102.50,143.33,-30.68,H-SC,9.73,105100.0,2600.0,38225.0,-0.54,2.54,36.37,39.83,114.0,0.07,0.78,4.06,AR,ATH,MISC


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,SIS,477.00,528.00,2240.93,H-SC,5.64,62985.0,-18105.0,26775.0,1.30,-22.33,42.51,10.69,156.0,-0.68,0.47,27.65,OX40N,NTT,MISC
6,ASIANTILES,75.41,137.00,6816.67,L-SC,6.81,75884.0,-17926.0,94544.0,-1.69,-19.11,124.59,81.67,269.0,-0.19,0.57,46.95,XR,NTT,CERAMICS
23,COFFEEDAY,59.14,80.00,-43.66,L-SC,25.79,83309.0,-30240.0,70288.0,-2.12,-26.63,84.37,35.27,268.0,-0.43,0.62,103.90,XR,NTT,HOTELS
3,ALKYLAMINE,2347.98,4546.37,1.04,H-SC,16.12,94020.0,-6943.0,101476.0,1.66,-6.88,107.93,93.63,148.0,-0.07,0.70,40.04,SR,ATH,CHEMICALS
54,MASFIN,326.60,399.50,-15.99,H-SC,12.70,95400.0,-2580.0,24451.0,1.31,-2.63,25.63,22.32,152.0,-0.11,0.71,38.36,XR,ATH,FINANCE


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-43.66,L-SC,25.79,83309.0,-30240.0,70288.0,-2.12,-26.63,84.37,35.27,268.0,-0.43,0.62,103.90,XR,NTT,HOTELS
35,HEROMOTOCO,4311.81,5949.07,-4.42,H-MC,13.13,176946.0,26033.0,31266.0,1.15,17.25,17.67,37.97,93.0,0.83,1.32,46.92,AR,ATH,AUTO
28,DMART,3764.92,5201.00,-7.19,H-LC,10.30,231770.0,47289.0,23084.0,0.66,25.63,9.96,38.14,38.0,2.05,1.73,38.96,X40N,NTT,FMCG
15,BLUESTARCO,1646.70,2326.38,11.16,H-MC,5.01,192000.0,27330.0,40646.0,0.00,16.60,21.17,41.28,89.0,0.67,1.43,25.90,X40N,ATH,AC
89,VBL,492.64,671.64,-8.30,H-LC,8.95,278630.0,13590.0,82725.0,0.25,5.13,29.69,36.33,5.0,0.16,2.08,19.11,X40N,ATH,FMCG


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.88
1,25,41.46
2,50,71.05


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    37.42
LC    36.28
MC    26.29
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.35
X40      14.39
X40N     13.99
XY25     11.80
XR        9.03
AR        8.46
OX40N     7.43
X5K       2.25
MH        1.73
X200      1.43
SR        1.13
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    28.07
H-MC    23.40
H-SC    22.66
M-SC    13.20
M-LC     7.16
M-MC     2.57
L-SC     1.56
L-LC     1.05
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.53,0.15,30.42
IT,12.03,-15.72,73.52
FINANCE,8.08,-20.41,64.96
BANKS,7.54,-12.54,60.32
MISC,5.88,-16.47,83.23
PAINTS,5.46,-19.12,42.60
ELECTRICAL,4.74,-10.40,58.22
AUTO,4.55,-10.96,59.24
AC,3.69,2.35,30.73


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2835716.0,23
XR,1082448.0,13
AR,1075221.0,10
X40,808599.0,11
X40N,490450.0,11
XY25,464417.0,8
OX40N,461688.0,10
SR,191634.0,2
X5K,136643.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2734881.0,27
M-SC,1476748.0,17
H-MC,1241060.0,18
H-LC,1138934.0,19
M-LC,418296.0,5
M-MC,333301.0,2
L-SC,252952.0,3
L-MC,59046.0,1
L-LC,44063.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      930501.0      6
M-SC       XY24      868591.0      7
H-SC       AR        654415.0      5
           XR        517914.0      6
H-LC       X40       512079.0      6
H-MC       XY24      486719.0      5
M-MC       XY24      333301.0      2
H-LC       AR        230813.0      2
H-MC       X40       213598.0      4
H-SC       OX40N     199098.0      4
H-MC       XY25      196776.0      2
H-SC       SR        191634.0      2
           X40N      186899.0      3
H-LC       X40N      184490.0      5
M-LC       XY24      169811.0      2
L-SC       XR        164832.0      2
M-SC       AR        158727.0      2
M-LC       X5K       129969.0      1
M-SC       OX40N     124390.0      4
           XY25      121800.0      1
           XR        120318.0      2
H-MC       X40N      119061.0      3
           XR        114093.0      1
M-LC       XR        106245.0      1
H-LC       XY25       89507.0      3
L-SC       OX40N      88120.0      1
M-SC       X40        82922.0      1
H-LC       X200       68578.0      1
L-MC       XR         59046.0      1
H-SC       MH         54420.0      1
H-MC       OX40N      50080.0      1
H-LC       XY24       46793.0      1
L-LC       XY25       44063.0      1
H-MC       AR         31266.0      1
           MH         29467.0      1
M-LC       XY25       12271.0      1
H-LC       X5K         6674.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
